In [1]:
import pandas as pd
import random, numpy

from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import time

In [2]:
CODEDIR = f"../../Code/KNAPSACK"
import sys
sys.path.append(CODEDIR)

from read_knapsack_files import read_knapsack
from greedy_knapsack_individual import knapsack_greedy

In [3]:
filename_1 = 'KNAPSACK files/knapPI_1_500_1000_1.txt'
filename_2 = 'KNAPSACK files/f2_l-d_kp_20_878.txt'
filename_3 = 'KNAPSACK files/f1_l-d_kp_10_269.txt'

In [7]:
def main(NGEN, MU, LAMBDA, CXPB, MUTPB, NPOP, NGREEDY, NREPS, filename, outputfilename):

    start = time.time()
    N, MAX_WEIGHT, items = read_knapsack(filename)
 


    #minimiza el peso y maximiza el valor
    creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))

    #representamos los individuos en una lista
    #creator.create("Individual", list, fitness=creator.Fitness)

    #representamos los individuos en un cjto
    creator.create("Individual", set, fitness=creator.Fitness)

    # Create the item dictionary: item name is an integer, and value is 
    # a (weight, value) 2-uple.
    #items = [(4, 12), (2, 1), (10, 4), (1, 1), (2, 2), (5, 1), (1, 1), (5, 1), (2, 3), (10, 4), (9, 1)]

    
    toolbox = base.Toolbox()
    toolbox.register("knapsack_greedy", knapsack_greedy, filename)
    toolbox.register("attr_item", random.randrange, N)  ## N = NBR_ITEMS
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
        toolbox.attr_item, N) ## N = IND_INIT_SIZE
    toolbox.register("individual2", tools.initIterate, creator.Individual, toolbox.knapsack_greedy)
    toolbox.register("population", tools.initRepeat, list)


    def evalKnapsack(individual):
        weight = 0.0
        value = 0.0
        for item in individual:
            weight += items[item][0]
            value += items[item][1]
        if weight > MAX_WEIGHT:
            return 10000, 0             # Ensure overweighted bags are dominated
        return weight, value

    #crossover in sets: producing two children from two parents, could be that the first
    #child is the intersection of the two sets and the second child their absolute difference
    def cxSet(ind1, ind2):
        """Apply a crossover operation on input sets. The first child is the
        intersection of the two sets, the second child is the difference of the
        two sets.
        """
        temp = set(ind1)                # Used in order to keep type
        ind1 &= ind2                    # Intersection (inplace)
        ind2 ^= temp                    # Symmetric Difference (inplace)
        return ind1, ind2

    #mutation operator could randomly add or remove an element from the set input individual
    def mutSet(individual):
        """Mutation that pops or add an element."""
        if random.random() < 0.5:
            if len(individual) > 0:     # We cannot pop from an empty set
                individual.remove(random.choice(sorted(tuple(individual))))
        else:
            individual.add(random.randrange(N)) ## N = IND_INIT_SIZE
        return individual,


    dict_result = dict()
    dict_time = dict()
    for i in range(NREPS):
        
        #register these operators in the toolbox
        toolbox.register("evaluate", evalKnapsack)
        toolbox.register("mate", cxSet)
        toolbox.register("mutate", mutSet)
        toolbox.register("select", tools.selNSGA2)
    
        pop1 = toolbox.population(toolbox.individual, n=NPOP)
        pop2 = toolbox.population(toolbox.individual2, n=NGREEDY)
        pop = pop1 + pop2
        
        hof = tools.ParetoFront()
        stats = tools.Statistics(lambda ind: ind.fitness.values[1])
        stats.register("avg", numpy.mean)
        stats.register("std", numpy.std)
        #stats.register("min", numpy.min)
        stats.register("max", numpy.max)
    
        pop, logbook = algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
        result_list = logbook
        end = time.time()
        dict_result[i] = result_list[-1]
        dict_time[i] = {'time': end-start}


    df1 = pd.DataFrame.from_dict(dict_result, orient = 'index')
    df2 = pd.DataFrame.from_dict(dict_time, orient = 'index')
    df = pd.concat([df1,df2], axis=1, sort=False)
    df.to_csv('Data_KNAPSACK/' + outputfilename + 'complete.csv', sep=';', header=True, index=True)
    df = df.describe()
    df.to_csv('Data_KNAPSACK/' + outputfilename + '.csv', sep=';', header=True, index=True)